# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/foohm/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/foohm/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [5]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [6]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/Users/foohm/AIMakerSpace/AIE7/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/Users/foohm/AIMakerSpace/AIE7/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/Users/foohm/AIMakerSpace/AIE7/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [7]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [8]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [9]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node 'b6e7d0'. Skipping!
Property 'summary' already exists in node 'd68222'. Skipping!
Property 'summary' already exists in node '8ad203'. Skipping!
Property 'summary' already exists in node '50d1ce'. Skipping!
Property 'summary' already exists in node 'e0bc17'. Skipping!
Property 'summary' already exists in node '976aae'. Skipping!
Property 'summary' already exists in node '906480'. Skipping!
Property 'summary' already exists in node 'fcdcc4'. Skipping!
Property 'summary' already exists in node '8d0a95'. Skipping!
Property 'summary' already exists in node 'b468a0'. Skipping!
Property 'summary' already exists in node 'eb7080'. Skipping!
Property 'summary' already exists in node '980053'. Skipping!
Property 'summary' already exists in node 'bd97e8'. Skipping!
Property 'summary' already exists in node 'a7f371'. Skipping!


Applying CustomNodeFilter:   0%|          | 0/6 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/43 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node 'b6e7d0'. Skipping!
Property 'summary_embedding' already exists in node 'e0bc17'. Skipping!
Property 'summary_embedding' already exists in node 'eb7080'. Skipping!
Property 'summary_embedding' already exists in node 'fcdcc4'. Skipping!
Property 'summary_embedding' already exists in node '976aae'. Skipping!
Property 'summary_embedding' already exists in node '50d1ce'. Skipping!
Property 'summary_embedding' already exists in node 'd68222'. Skipping!
Property 'summary_embedding' already exists in node 'a7f371'. Skipping!
Property 'summary_embedding' already exists in node '8ad203'. Skipping!
Property 'summary_embedding' already exists in node '906480'. Skipping!
Property 'summary_embedding' already exists in node '8d0a95'. Skipping!
Property 'summary_embedding' already exists in node 'b468a0'. Skipping!
Property 'summary_embedding' already exists in node '980053'. Skipping!
Property 'summary_embedding' already exists in node 'bd97e8'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 40, relationships: 478)

We can save and load our knowledge graphs as follows.

In [10]:
kg.save("loan_data_kg.json")
loan_data_kg = KnowledgeGraph.load("loan_data_kg.json")
loan_data_kg

KnowledgeGraph(nodes: 40, relationships: 478)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [11]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loan_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [13]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


#### Answer #1:

See my query to Cursor in PROMPTS.md (Prompt 6) where I clarified my understanding of each of the Query Synthesizers

Essentially there are 2 main concepts:

1. Single Hop vs Multi Hop queries

Single Hop means number of data pieces eg. chunks needed to answer the query. Multi Hop means the need to combine more than 1 chunk of information. 

2. Specific vs Abstract

The answer is specific in the data eg. minimum number of weeks for credit-hour. Abstract means there is some reasoning, synthesis, clarification needed (hence not direct) eg. why would a school request a reduction in minimum weeks in instructional time?

Finally, we can use our `TestSetGenerator` to generate our testset!

In [15]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,Can you explain how the term Tittle IV is rela...,"[Chapter 1 Academic Years, Academic Calendars,...",The context does not explicitly define 'Title ...,single_hop_specifc_query_synthesizer
1,What does 34 CFR 668.3(a) specify regarding th...,[Regulatory Citations Academic year minimums: ...,Regulatory Citations Academic year minimums: 3...,single_hop_specifc_query_synthesizer
2,Whaat is Volum 8 in the contex of standard and...,[Inclusion of Clinical Work in a Standard Term...,Inclusion of Clinical Work in a Standard Term ...,single_hop_specifc_query_synthesizer
3,Is the Federal Work-Study (FWS) program subjec...,[Non-Term Characteristics A program that measu...,"No, the Federal Work-Study (FWS) program is an...",single_hop_specifc_query_synthesizer
4,Can you tell me about Volume 8 and its relevan...,[both the credit or clock hours and the weeks ...,The provided context discusses Volume 8 in rel...,single_hop_specifc_query_synthesizer
5,"How do variation for term-based, non-term, clo...","[<1-hop>\n\nChapter 1 Academic Years, Academic...","The variation for term-based, non-term, clock-...",multi_hop_abstract_query_synthesizer
6,How do the disbursement timing requirements fo...,[<1-hop>\n\nboth the credit or clock hours and...,The context explains that for programs with no...,multi_hop_abstract_query_synthesizer
7,How does the academic year based on instructio...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The academic year defined by instructional tim...,multi_hop_abstract_query_synthesizer
8,How do the disbursement timing rules outlined ...,[<1-hop>\n\nDisbursement Timing in Subscriptio...,The disbursement timing rules in Appendix B pr...,multi_hop_specific_query_synthesizer
9,Volume 2 and Volume 8 include clinical work an...,[<1-hop>\n\nInclusion of Clinical Work in a St...,Volume 2 discusses inclusion of clinical work ...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [16]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '24f1d3'. Skipping!
Property 'summary' already exists in node '147b2f'. Skipping!
Property 'summary' already exists in node 'd8479b'. Skipping!
Property 'summary' already exists in node 'aae036'. Skipping!
Property 'summary' already exists in node '6314b5'. Skipping!
Property 'summary' already exists in node '51e77c'. Skipping!
Property 'summary' already exists in node 'e01d87'. Skipping!
Property 'summary' already exists in node 'ac410d'. Skipping!
Property 'summary' already exists in node '77408e'. Skipping!
Property 'summary' already exists in node '5113ce'. Skipping!
Property 'summary' already exists in node 'cd34a6'. Skipping!
Property 'summary' already exists in node 'a2dad1'. Skipping!
Property 'summary' already exists in node 'f45263'. Skipping!
Property 'summary' already exists in node 'f7977c'. Skipping!


...

Property 'summary_embedding' already exists in node '24f1d3'. Skipping!
Property 'summary_embedding' already exists in node 'aae036'. Skipping!
Property 'summary_embedding' already exists in node 'ac410d'. Skipping!
Property 'summary_embedding' already exists in node '6314b5'. Skipping!
Property 'summary_embedding' already exists in node 'cd34a6'. Skipping!
Property 'summary_embedding' already exists in node 'd8479b'. Skipping!
Property 'summary_embedding' already exists in node '147b2f'. Skipping!
Property 'summary_embedding' already exists in node '5113ce'. Skipping!
Property 'summary_embedding' already exists in node 'e01d87'. Skipping!
Property 'summary_embedding' already exists in node '51e77c'. Skipping!
Property 'summary_embedding' already exists in node 'a2dad1'. Skipping!
Property 'summary_embedding' already exists in node '77408e'. Skipping!
Property 'summary_embedding' already exists in node 'f7977c'. Skipping!
Property 'summary_embedding' already exists in node 'f45263'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [21]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,How does the FAFSA process incorporate the FAF...,[Application and Verification Guide Introducti...,The FAFSA process has been overhauled by the F...,single_hop_specifc_query_synthesizer
1,Can you tell me about the FAFSA and what happe...,[Chapter 1: The Application Process We removed...,Chapter 1 states that the Returning FAFSA File...,single_hop_specifc_query_synthesizer
2,What is Federal Tax Information used for in FA...,[electronically due to limitations on access t...,The context does not explicitly explain what F...,single_hop_specifc_query_synthesizer
3,What is the link to the 2024-25 FAFSA Partner ...,[Note: There will be two active portals that c...,The link to the 2024-25 FAFSA Partner Portal i...,single_hop_specifc_query_synthesizer
4,how returning FAFSA filers do they need to rea...,[<1-hop>\n\nApplication and Verification Guide...,The context indicates that the section on Retu...,multi_hop_abstract_query_synthesizer
5,How does the removal of COVID-19 related guida...,[<1-hop>\n\nApplication and Verification Guide...,The removal of COVID-19 related guidance has l...,multi_hop_abstract_query_synthesizer
6,How does the use of IRS data for FAFSA eligibi...,[<1-hop>\n\nApplication and Verification Guide...,The use of IRS data for FAFSA eligibility dete...,multi_hop_abstract_query_synthesizer
7,How do restrictions on FAFAs initiating new ap...,[<1-hop>\n\nelectronically due to limitations ...,Restrictions on FAFAs initiating new applicati...,multi_hop_abstract_query_synthesizer
8,"In chapter 2, student family size example and ...",[<1-hop>\n\nStudent Citizenship (13) Family Si...,"According to Chapter 2, the student can includ...",multi_hop_specific_query_synthesizer
9,How do Chapters 2 and 5 relate to determining ...,[<1-hop>\n\nStudent Citizenship (13) Family Si...,Chapter 2 provides detailed examples of how fa...,multi_hop_specific_query_synthesizer


##### Comment #1

Only after stepping through the code did I understand the big picture. We could effectively start from the Abstracted SDG to generate the test data set but to understand why and what actually happens under the hood we would have to go through building the KG, creating the query synthesizers etc. That was great!

We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [17]:
from langsmith import Client

client = Client()

dataset_name = "Loan Synthetic Data"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Loan Synthetic Data"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [18]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [19]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [21]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [22]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan RAG"
)

In [23]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [24]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [25]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [26]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [27]:
rag_chain.invoke({"question" : "What kinds of loans are available?"})

'The kinds of loans available include:\n\n- Direct PLUS Loan or student Federal PLUS Loan\n- Subsidized Federal Stafford Loans\n- Unsubsidized Federal Stafford Loans\n- Federal SLS Loans\n- Federal PLUS Loans\n- Direct Subsidized Loans\n- Direct Unsubsidized Loans\n\nNote: Subsidized and Unsubsidized Federal Stafford Loans, Federal SLS Loans, and Federal PLUS Loans were made under the Federal Family Education Loan (FFEL) Program before July 1, 2010, when the authority to make new FFEL Program loans ended. Direct Subsidized Loans, Direct Unsubsidized Loans, and Direct PLUS Loans are current loan types under the Direct Loan Program.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [28]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [29]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `empathy_evaluator`:

The answer is found in Prompts 8, 9, 10

1. The QA evaluator 

It compares the output of the RAG application with "ground truth" which could be handcrafted or synthetic. In this case it compares the Outputs with the Reference Outputs. Most likely it uses a similarity metric like cosine similarity to compare the 2 and get a result. 

2. Helpfulness evaluator

This is using the "labeled_criteria" evaluator with the criteria whether the response is "helpful" or not. I did ask Cursor to clarify this in Prompt 9 and essentially it is using the LLM's "understanding" of helpfulness since we are using LLM as a judge. We can actually be more specific by providing more context in the definition of the evaluator eg. assume that the user is a student hence we want the response to be helpful to a student. 

3. Empathy evaluator

This is a "criteria" evaluator with the criteria of whether the response is empathetic or not. Same issue I had with the helpfulness evaluator cos how do you define "empathetic". Since we're using LLM as a judge we rely its "understanding" of that word. 

## LangSmith Evaluation

In [30]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'monthly-corn-89' at:
https://smith.langchain.com/o/8edb76a9-4b6c-4059-bd8e-6002b94e5ffc/datasets/3ab2e497-a161-4044-ab52-ef19eeebf7d5/compare?selectedSessions=3a9f180d-ace7-4398-82da-34bd84ca0f06




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do Volume 2 and Volume 8 relate to the req...,"Based on the provided context:\n\n- Volume 2, ...",None,Volume 2 discusses the effect of accelerated p...,0,0,0,5.186978,6f54e848-35a5-4813-84e7-9c7d0aed0ca4,bf8ce14a-f69b-492c-aa22-c162f8a87d36
1,How do Appendix A and Appendix B provide guida...,Based on the context provided:\n\nAppendix B i...,None,Appendix B offers detailed guidance on disburs...,0,0,0,6.854640,1a5a4a98-ecad-42d5-9728-877b4f537e23,95ba4a7e-6842-4bba-aa56-626da76aaf9e
2,How does Volume 8 address the inclusion of cli...,"Volume 8, Chapter 3 provides additional guidan...",None,Volume 8 explains that clinical work conducted...,1,0,0,2.736799,9a91881a-35dc-47bb-9ffb-f7ab1987fb15,b4bc917f-f4cd-4383-bb27-a49f29494a1c
3,How do Chapters 2 and 3 relate to the manageme...,"Based on the provided context, Chapters 2 and ...",None,Chapter 2 discusses the inclusion of clinical ...,1,1,0,9.135171,141d8d19-7fd4-43f8-88c3-cc49642e96f8,74d2b4d4-f379-4bf1-9a8b-bf22e383992e
4,clinical work overlap flexible term classifica...,If the clinical work meets all the required cr...,None,The context explains that clinical work conduc...,1,1,0,3.936683,269b84a3-77cc-46ae-90b6-d77eaf60f1ce,1b623939-4f2d-45a8-8c92-b9a81facfb24
5,How do disbursement timing requirements differ...,Based on the provided context:\n\nFor **clock-...,None,In clock-hour or non-term credit-hour programs...,1,1,0,7.760928,fa24abfc-68b9-4cd6-aee1-29564ef19a5a,8e443a52-b5c2-4dd0-bda0-b0f08fddbaf5
6,impact of term structure on academic progress ...,The term structure impacts academic progress m...,None,The context explains that inclusion of clinica...,1,1,0,3.675220,ad792d16-fedf-4480-97ad-4008ce550cae,1001779c-0ba9-43be-9f52-a87ccfa200b3
7,Credit hour allocation for clinical experience...,"Based on the provided context, clinical work c...",None,The context explains that clinical work includ...,1,1,0,4.131777,947e5747-79fb-4583-b2fd-63850e3fffd3,f98a9eaf-30c2-4142-803b-37df25d4f1ac
8,What is the significance of Volume 8 in relati...,"Volume 8 of the Federal Student Aid Handbook, ...",None,Volume 8 discusses the effect of accelerated p...,0,0,0,2.400219,d8146032-90b1-45ab-aa3e-7b250b3c0289,57268145-6927-4293-80e5-d7df15e3c6a6
9,Could you explain the significance of Title IV...,"Based on the provided context, Title IV progra...",None,"Title IV programs, except for the Federal Work...",1,1,0,4.844896,c28739c7-2226-46f9-97d5-aeb0f0e64936,f0fd95f9-478e-4971-a66f-c0a42b207d24


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [31]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [32]:
rag_documents = docs

In [33]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

#### Answer #2:

To understand this question we need to first understand 2 key concepts:
1. Why do we do chunking?
2. What do we mean by "performance of our application" in this context?

**Why do we do chunking** 
- the ultimate goal is to be able to feed the LLM with the right information in the context window so that it has the necessary information to make inferences 
- the context window is of limited size
- focus: if fed information not relevant to the query the LLM could hallucinate 
- in a large corpus of domain information espressed as text, we need a way to store chunks of text that are "information cohesive" ie. it talks about one or very few concepts or ideas 
- when we have the domain information organized in this way we can then find all the chunks relevant to the query and out them in the context window 

**What is "performance"**
- in the context of RAG applications, performance means how "well" the RAG application is able to answer the question
- measures like correctness, relevance, faithfulness, helpfulness are used 
- in the context of answering the student loan question a measure like empathy could also be linked to performance

**Some impacts of chunk size**
- if the chunk size is too small (eg. 5 words), we are likely not capturing sufficient relevant information to be useful
- if the chunk size is too large, it could span several key ideas/concepts ie. not "information cohesive" 
- if the chunk size is too large, it could also lead to the "Lost in the middle" issue where a key idea is buried in the chunk and its relevance to the query gets muddied by the surrounding text and hence is lower down the result list of the similarity search
- if the chunk size is too large, we may run out of context window space before all the relevant chunks are included
- if the chunk size is too large, you may need to truncate it to fit into the embedding size limit hence losing information

Hence to answer the question: in order for the RAG application to effecively answer queries ie. performance, it needs the relevant information in the context and hence the size of the chunk matters. There are several other ways to chunk to achieve the same goal other than RecursiveCharacterTextSplitter. Some corpuses of domain information are naturally chunked eg. bug descriptions, TOC in documents.  

In [35]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

#### Answer #3:

1. See previous answer for "What is performance"
2. See previous answer for how chunk size affects performance

There are 2 aspects of the embedding model that are key here:
1. The size (or number of dimensions) of the embedding model ie. size of the vector space used to encode the tokens
2. The embedding algorithm ie. how chunks of text are mapped to the vector space 

**To answer the question**
- With larger sizes of the embedding model, we are able to capture more "meaning" of the text and hence the intuition is that when we retrieve the relevant chunks using similarity metrics eg. cosine similarity we get more relevant chunks and hence the context is populated with more of relevant information for the LLM to answer the query
- to some extent whether the embedding model is able to exploit the larger size also depends on the embedding algorithm. 
- With the increased size of the chunks, if the embedding model is too small we could run into an issue where some of the chunks need to be truncated in the embedding hence losing information  

In [36]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan Data for RAG"
)

In [37]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [38]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [39]:
empathy_rag_chain.invoke({"question" : "What kinds of loans are available?"})

"Thank you for your question. Based on the information provided, there are several types of loans available to students and their parents to help cover the cost of attendance:\n\n1. **Direct Subsidized Loans** – These loans are based on the student's financial need. They help cover part of the cost of attendance, and the government generally pays the interest while the student is in school.\n\n2. **Direct Unsubsidized Loans** – These loans are not based on financial need and are available to both dependent and independent students. Interest accrues during all periods.\n\n3. **Direct PLUS Loans** – These are loans that parents of dependent students can take out to help pay for the student's cost of attendance, provided they meet eligibility requirements. If a parent can't obtain a PLUS loan, the student may be eligible for additional unsubsidized loans.\n\n4. **Combination Options** – A student may qualify for a combination of Direct Subsidized and Direct Unsubsidized Loans depending on

Finally, we can evaluate the new chain on the same test set!

In [40]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'sunny-writer-84' at:
https://smith.langchain.com/o/8edb76a9-4b6c-4059-bd8e-6002b94e5ffc/datasets/3ab2e497-a161-4044-ab52-ef19eeebf7d5/compare?selectedSessions=80e80e3c-5dfd-43c0-a285-88e195da52e0




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do Volume 2 and Volume 8 relate to the req...,Thank you for your thoughtful question. From t...,None,Volume 2 discusses the effect of accelerated p...,1,0,1,5.085310,6f54e848-35a5-4813-84e7-9c7d0aed0ca4,d9502e6d-4d87-4478-b1b3-c6859cac72c8
1,How do Appendix A and Appendix B provide guida...,Thank you for your thoughtful question. From t...,None,Appendix B offers detailed guidance on disburs...,0,0,1,4.537161,1a5a4a98-ecad-42d5-9728-877b4f537e23,bd46ecdc-fe52-44aa-83b7-5bfbd613c502
2,How does Volume 8 address the inclusion of cli...,Thank you for your thoughtful question. Based ...,None,Volume 8 explains that clinical work conducted...,0,0,1,3.566933,9a91881a-35dc-47bb-9ffb-f7ab1987fb15,7206293f-a045-4538-88e4-30e726651641
3,How do Chapters 2 and 3 relate to the manageme...,Thank you for your thoughtful question. It’s c...,None,Chapter 2 discusses the inclusion of clinical ...,1,1,1,9.884570,141d8d19-7fd4-43f8-88c3-cc49642e96f8,50a9cb7f-fb69-42dc-83ca-6e59cb750e7e
4,clinical work overlap flexible term classifica...,Thank you for your thoughtful question. Based ...,None,The context explains that clinical work conduc...,1,1,1,6.033928,269b84a3-77cc-46ae-90b6-d77eaf60f1ce,af28a90b-a4ad-4677-88ce-c25e2e235ec9
5,How do disbursement timing requirements differ...,Thank you for your thoughtful question—I under...,None,In clock-hour or non-term credit-hour programs...,1,1,1,114.575722,fa24abfc-68b9-4cd6-aee1-29564ef19a5a,09f398b4-f8a3-4a70-9491-e27384b6570d
6,impact of term structure on academic progress ...,Thank you for your thoughtful question about h...,None,The context explains that inclusion of clinica...,1,1,1,10.211071,ad792d16-fedf-4480-97ad-4008ce550cae,30dd43ff-0167-482a-94e3-f8e012d88afe
7,Credit hour allocation for clinical experience...,Thank you for your thoughtful question. From t...,None,The context explains that clinical work includ...,1,1,1,3.641948,947e5747-79fb-4583-b2fd-63850e3fffd3,e797284c-8a6f-4678-83cd-fadcb886a53f
8,What is the significance of Volume 8 in relati...,Thank you for your thoughtful question about V...,None,Volume 8 discusses the effect of accelerated p...,0,0,1,5.672466,d8146032-90b1-45ab-aa3e-7b250b3c0289,de731cb2-5e74-4e03-8ec5-c3bc64dc7ae1
9,Could you explain the significance of Title IV...,Thank you for your thoughtful question about T...,None,"Title IV programs, except for the Federal Work...",1,1,1,5.816190,c28739c7-2226-46f9-97d5-aeb0f0e64936,e9ca0b22-cedc-4c8f-971d-824a4d6a00a1


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

![Screenshot of Langsmith eval](Langsmith-20250716.png)

1. If you look at all 3 evals, only the Empathy metric has changed and 180 degrees too ie. the responses are not more or less correct or helpful
2. I took a look are more detail into how Langsmith performed the Empathy evals and it looks for things like the following in the response:
    - acknowledging the user's particular concerns or feelings
    - the user's perspective is being considered
    - summary does not reflect or tailor itself receptively to the user's needs
    - no language in the submission that demonstrates warmth, relates to the user's possible confusion, or communicates that the answerer is particularly invested in helping the user
3. If we compare it to the prompt used it includes "You must answer the question using empathy and kindness, and make sure the user feels heard." it seems to have provided the LLM sufficient context to generate the responses to meet the needs of the empathy evaluator.
4. When digging into the new set of responses, I notice common sentances in the responses hence meeting the needs of the Empathy evaluator
    - Thank you for your thoughtful question
    - I truly understand that navigating these details can feel complex, and I hope this explanation helps (or similar message)
    - I’m here to help! (or similar message)